In [124]:
##Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from IPython.display import clear_output
##Import bq_helper to pull forestry data from Kaggle
from bq_helper import BigQueryHelper #pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
import os
#Setup Google Credentials(Data is stored on Google Cloud Servers)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="ErdosTrees-f85df8c8cb95.json" #Get .json file from Google Cloud Platform
import time

### Get Tree Data

In [125]:
#Calling the FIA BigQuery API
usfs = BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="usfs_fia")

In [126]:
#Defining the states in the southern region. We could expand it to include other states
states =('Alabama', 'Arkansas', 'Florida', 
         'Georgia', 'Kentucky', 'Louisiana', 
         'Mississippi', 'North Carolina', 
         'Oklahoma', 'South Carolina', 'Tennessee', 
         'Texas', 'Virginia')

Species = pd.read_csv('Top5_Species.csv',index_col = 0)
regions = pd.read_csv('State_Region.csv',index_col = 0)

In [128]:
f1 = pd.DataFrame([['a', 1], ['b', 2]],columns=['letter', 'number'])
f2 = pd.DataFrame([['c', 3], ['d', 4]],columns=['letter', 'number'])
f1.append(f2)

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [141]:
#Defining the query for what data I want from the FIA API
#This gets the top 10 trees per plot
region = 2 #southern region
states = tuple(regions.index[regions.state_region == region])
species = tuple(Species.loc[region].values)
for k,s in enumerate(species):
    query1 = f"""
            SELECT
                plot_sequence_number AS id,
                species_common_name AS species,
                plot_state_code AS state_code,
                plot_county_code AS county_code,
                measurement_year AS year,
                latitude,
                longitude,
                SUM(gross_cubicfoot_volume) AS volume,
                SUM(aboveground_carbon) AS Biomass,
                COUNT(species_common_name) AS number_trees
            FROM(
                SELECT
                    plot_sequence_number,
                    species_common_name,
                    plot_state_code,
                    plot_county_code,
                    measurement_year,
                    latitude,
                    longitude,
                    gross_cubicfoot_volume,
                    aboveground_carbon,
                    ROW_NUMBER() 
                        OVER(
                            PARTITION BY plot_sequence_number 
                            ORDER BY aboveground_carbon DESC) AS rid
                FROM
                    `bigquery-public-data.usfs_fia.plot_tree`
                WHERE
                    plot_state_code IN {states}
                    AND measurement_year BETWEEN 2001 AND 2019
                    AND species_common_name = "{s}"
                    AND aboveground_carbon > 0)
                WHERE 
                    rid <= 10
                GROUP BY
                    plot_sequence_number,
                    plot_state_code,
                    plot_county_code,
                    measurement_year,
                    latitude,
                    longitude,
                    species_common_name
                ORDER BY
                    plot_state_code,
                    plot_county_code,
                    species_common_name,
                    measurement_year,
                    plot_sequence_number
            ;
                    """
    #Creating a pandas dataframe for the data requested
    if k == 0:
        plots = usfs.query_to_pandas_safe(query1, max_gb_scanned=10)
    else:
        plots = plots.append(usfs.query_to_pandas_safe(query1, max_gb_scanned=10))
plots = plots.reset_index(drop=True)

In [4]:
# DO NOT RUN THIS IF YOU RAN THE CELL ABOVE, IT IS ONE OR THE OTHER!!!

#Defining the query for what data I want from the FIA API
#This gets the volume/biomass for all trees of a species in a plot
region = 2 #southern region
states = tuple(regions.index[regions.state_region == region])
species = tuple(Species.loc[region].values)

query1 = f"""
        SELECT
            plot_sequence_number,
            species_common_name AS species,
            plot_state_code AS state_code,
            plot_county_code AS county_code,
            measurement_year AS year,
            latitude,
            longitude,
            SUM(gross_cubicfoot_volume) AS volume,
            SUM(aboveground_carbon) AS Biomass,
            COUNT(species_common_name) AS number_trees 
        FROM
            `bigquery-public-data.usfs_fia.plot_tree`
        WHERE
            plot_state_code IN {states}
            AND measurement_year BETWEEN 2001 AND 2019
            AND species_common_name IN {species}
            AND aboveground_carbon > 0
        GROUP BY
             plot_sequence_number,
             plot_state_code,
             plot_county_code,
             measurement_year,
             latitude,
             longitude,
             species_common_name
        ORDER BY
            plot_state_code,
            plot_county_code,
            species_common_name,
            measurement_year,
            plot_sequence_number
        ;
                """
#Creating a pandas dataframe for the data requested
plots = usfs.query_to_pandas_safe(query1, max_gb_scanned=10)
plots = plots.reset_index(drop=True)

### Get Station Data

In [143]:
#Calling the NOAA GSOD BigQuery API 
noaa_gsod = BigQueryHelper(active_project= "bigquery-public-data", 
                                     dataset_name= "noaa_gsod")

In [144]:
#defining the query to grab the station table
query1 = """
            SELECT 
                usaf AS Station_number, 
                lat AS Latitude, 
                lon AS Longitude, 
            FROM 
                `bigquery-public-data.noaa_gsod.stations` 
            WHERE 
                country = 'US' AND lat IS NOT NULL AND lon IS NOT NULL AND NOT (lat = 0.0 AND lon = 0.0) AND NOT usaf = '999999' 
        """

In [145]:
#Creating a dataframe from the station numbers. Some of the stations are in the same location (lat and long).
#Dropped these excess stations
stations1 = noaa_gsod.query_to_pandas_safe(query1, max_gb_scanned=10)
stations = stations1.copy()
stations = stations.drop(stations.loc[stations.Station_number.duplicated(keep='last')].index)

### Assemble plot data with climate data

In [147]:
def gsodqueryyear(year,stations, db):
    var2 = 'bigquery-public-data.noaa_gsod.gsod'+str(year)
    query = f"""
            SELECT 
                stn AS Station_number, 
                year AS Year, 
                AVG(temp) as Mean_temp,
                AVG(dewp) as Mean_dwp,
                AVG(prcp) as Mean_prcp
            FROM 
                {var2} 
            WHERE 
                Stn in {stations} AND NOT dewp = 9999.9 AND NOT prcp = 99.99 AND NOT temp = 9999.9
            GROUP BY
                stn, 
                year 
        """
    return db.query_to_pandas_safe(query, max_gb_scanned=10)

#Defined a function that matches the stations in the gsod data with latitude and longitude
def latlong(gsod,stations):
    
    lat = np.empty(len(gsod.Station_number))
    long = np.empty(len(gsod.Station_number))
    gsod_copy = gsod.copy()
    for i in range(len(gsod.Station_number)):
        lat[i] = np.array(stations.loc[stations.Station_number == gsod.Station_number[i]].Latitude)[0]
        long[i] = np.array(stations.loc[stations.Station_number == gsod.Station_number[i]].Longitude)[0]
    gsod_copy['Latitude'] = lat
    gsod_copy['Longitude'] = long
    return gsod_copy

#Defined a function that links the plot locations in the data with the nearest GSOD station
def find_nearest3(lat,long,df):
    
    index_nearest = np.sqrt((lat-df.Latitude)**2 + (long-df.Longitude)**2).idxmin()
    return df.Station_number[index_nearest]

#Defined a function that applies the find_nearest3 to every plot in the plot dataframe
def get_station3(sta, plt):
    
    plt_copy = plt.copy()
    nstation = np.empty(len(plt_copy)).astype(str)
    nstation_ind = np.empty(len(plt_copy))
    
    for i in range(len(plt_copy)):
        
        nstation[i] = find_nearest3(round(plt_copy.latitude[i],3),round(plt_copy.longitude[i],3),sta)
        
    plt_copy['nearest_station'] = nstation
    
    return plt_copy

#Defined a function that adds the temperature, dewpoint, and pressure at the station nearest each plot
def add_temp(df,gsod):
    feats = np.empty((len(df),3))
    for i in range(len(df)):
        features = np.array(gsod.loc[(gsod.Station_number == df.nearest_station[i])][['Mean_temp','Mean_dwp','Mean_prcp']])
        
        #if there is no feature data available at the station, it will put in place a NaN 
        try:
            feats[i][0] =features[0][0]
        except:
            feats[i][0] = np.nan
        try:
            feats[i][1] =features[0][1]
        except:
            feats[i][1] = np.nan
        try:
            feats[i][2] =features[0][2]
        except:
            feats[i][2] = np.nan
    df = df.assign(mean_temp = feats[:,0], mean_dwp = feats[:,1], mean_prcp = feats[:,2])
    return df


In [148]:
#Creating a dataframe to store the plot data with added GSOD station features
plot_stations = pd.DataFrame()
stations_tuple = tuple(stations.Station_number)

#putting it all together
t0 = time.perf_counter()
for year in range(2001,2020):
    print(f'Working on {year}')
    plot_year = plots.loc[plots.year == year].reset_index(drop=True)
    gsod = gsodqueryyear(year, stations_tuple, noaa_gsod)
    gsod.Year = gsod.Year.astype(float)
    gsod = latlong(gsod,stations)
    plot_station = get_station3(gsod, plot_year)
    plot_station = add_temp(plot_station,gsod)
    plot_stations = pd.concat([plot_stations,plot_station])

t1 = time.perf_counter()
print(f'Runtime: {t1-t0} seconds')

Working on 2001
Working on 2002
Working on 2003
Working on 2004
Working on 2005
Working on 2006
Working on 2007
Working on 2008
Working on 2009
Working on 2010
Working on 2011
Working on 2012
Working on 2013
Working on 2014
Working on 2015
Working on 2016
Working on 2017
Working on 2018
Working on 2019
Runtime: 622.0657965999999 seconds


In [149]:
county_tree_df = plot_stations.drop(['id','nearest_station'],axis=1)
county_tree_df = county_tree_df.groupby(['year','state_code','county_code','species'])
county_tree_df = county_tree_df.agg({'latitude':'mean', 
                                     'longitude':'mean',
                                     'volume':'sum',
                                     'Biomass':'sum',
                                     'number_trees':'sum',
                                     'mean_temp':'mean',
                                     'mean_dwp':'mean',
                                     'mean_prcp':'mean',})
county_tree_df

latitude  longitude      volume  \
year state_code county_code species                                           
2001 1          1           loblolly pine  32.578953 -86.593001  254.208304   
                            red maple      32.620470 -86.587013    7.226856   
                            slash pine     32.515568 -86.626175    0.000000   
                            sweetgum       32.569998 -86.627121  161.481322   
                            yellow-poplar  32.638274 -86.539808   35.165761   
...                                              ...        ...         ...   
2019 48         457         red maple      30.901363 -94.226028    0.000000   
                            sweetgum       30.901363 -94.226028   63.029182   
                471         loblolly pine  30.759488 -95.662211  676.888912   
                            sweetgum       30.781962 -95.594315   12.674485   
                473         loblolly pine  30.154566 -95.804222   80.081194   

                                                Biomass  number_trees  \
year state_code county_code species                                     
2001 1          1           loblolly pine   5270.671429            50   
                            red maple        182.733664             3   
                            slash pine        16.737962             6   
                            sweetgum        3499.375783            17   
                            yellow-poplar    806.265958             8   
...                                                 ...           ...   
2019 48         457         red maple         96.761185            10   
                            sweetgum         864.124593            10   
                471         loblolly pine  12976.998329            21   
                            sweetgum         363.300493            10   
                473         loblolly pine   1585.137314             3   

                                           mean_temp   mean_dwp  mean_prcp  
year state_code county_code species                                         
2001 1          1           loblolly pine  67.470830  54.867025   0.153781  
                            red maple      68.050000  54.900915   0.158415  
                            slash pine     68.050000  54.900915   0.158415  
                            sweetgum       66.428324  54.806024   0.145440  
                            yellow-poplar  68.050000  54.900915   0.158415  
...                                              ...        ...        ...  
2019 48         457         red maple      65.456250  56.725000   0.000000  
                            sweetgum       65.456250  56.725000   0.000000  
                471         loblolly pine  68.726519  57.909392   0.098343  
                            sweetgum       68.726519  57.909392   0.098343  
                473         loblolly pine  69.121370  59.109041   0.138548  

[45365 rows x 8 columns]

In [150]:
#saving the results
county_tree_df.to_csv('county_tree_temp_dwp_prcp_KT.csv')